In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/college_project/ACDC/estimators")
#os.listdir()

In [0]:
import argparse
import os, sys, shutil
import numpy as np 
import time
from estimator import *
from config import *
sys.path.insert(0,'../models/')
from network import *
from network_ops import *


In [0]:
import matplotlib
matplotlib.use('TKAgg')


In [0]:
pip install SimpleITK

In [0]:
!pip install tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 61kB/s 
     |████████████████████████████████| 3.1MB 57.0MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


In [0]:


if __name__ == '__main__':
    conf = conf()
    inputs = tf.placeholder(tf.float32, shape=(None, None, None, conf.num_channels))
    targets = tf.placeholder(tf.uint8, shape = (None, None, None))
    weight_maps = tf.placeholder(tf.float32, shape=[None, None, None])
    batch_class_weights = tf.placeholder(tf.float32)

    # define the net
    print('Defining the network', conf.run_name)
    # model = FCMultiScaleResidualDenseNet(inputs,
    #                         targets, 
    #                         weight_maps,
    #                         batch_class_weights,
    #                         num_class=conf.num_class,
    #                         n_pool = 3, 
    #                         n_feat_first_layer = [12, 12, 12], 
    #                         growth_rate = 12,
    #                         n_layers_per_block = [2, 3, 4, 5, 4, 3, 2], 
    #                         weight_decay = 5e-6, 
    #                         dropout_rate = 0.2, 
    #                         optimizer = AdamOptimizer(conf.learning_rate),
    #                         metrics_list = ['sW_CE_loss', 'mBW_Dice_loss', 'L2_loss', 'Total_loss', 'avgDice_score',
    #                                         'Dice_class_1', 'Dice_class_2', 'Dice_class_3'],
    #                         metrics_to_optimize_on = ['Total_loss']
    #                       )

    model = FCMultiScaleResidualDenseNet(inputs,
                            targets, 
                            weight_maps,
                            batch_class_weights,
                            num_class=conf.num_class,
                            n_pool = 3, 
                            n_feat_first_layer = [16, 16, 16], 
                            growth_rate = 16,
                            n_layers_per_block = [2, 3, 4, 5, 4, 3, 2], 
                            weight_decay = 5e-6, 
                            dropout_rate = 0.2, 
                            optimizer = AdamOptimizer(conf.learning_rate),
                            metrics_list = ['sW_CE_loss', 'mBW_Dice_loss', 'L2_loss', 'Total_loss', 'avgDice_score',
                                            'Dice_class_1', 'Dice_class_2', 'Dice_class_3'],
                            metrics_to_optimize_on = ['Total_loss']
                          )

    # initialise the estimator with the net
    print('Preparing the estimator..')
    trainer = Estimator(model = model,
                        conf = conf
                        )
    # iterate for the number of epochs
    for epoch in range(int(trainer.numKeeper.counts['epoch']+1), conf.num_epochs):
        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
        print('Training @ Epoch : ' + str(epoch))
        trainer.Fit(steps=-1)
        print('\n---------------------------------------------------')
        print('Validating @ Epoch : ' + str(epoch))
        trainer.Evaluate(steps=-1 )
        trainer.numKeeper.counts['epoch'] = trainer.numKeeper.counts['epoch'] + 1
        trainer.numKeeper.UpdateCounts(trainer.summary_manager.counts)
        trainer.SaveModel(os.path.join(conf.output_dir,conf.run_name,'models','latest.ckpt'))






Defining the network FCRD_ACDC
First Layer shape  [None, None, None, 48]
DB_Down: 0  shape  [None, None, None, 80]
DB Projection Layer:  shape  [None, None, None, 48]
TD: 0  shape  [None, None, None, 80]
DB_Down: 1  shape  [None, None, None, 128]
DB Projection Layer:  shape  [None, None, None, 64]
TD: 1  shape  [None, None, None, 128]
DB_Down: 2  shape  [None, None, None, 192]
DB Projection Layer:  shape  [None, None, None, 80]
TD: 2  shape  [None, None, None, 192]
Bottleneck Projection Layer:  shape  [None, None, None, 80]
Bottleneck:  shape  [None, None, None, 80]
TU: 0  shape  [None, None, None, 80]
Transition Up Projection Layer:  shape  [None, None, None, 64]
DB_Up: 0  shape  [None, None, None, 64]
TU: 1  shape  [None, None, None, 64]
Transition Up Projection Layer:  shape  [None, None, None, 48]
DB_Up: 1  shape  [None, None, None, 48]
TU: 2  shape  [None, None, None, 48]
Transition Up Projection Layer:  shape  [None, None, None, 32]
DB_Up: 2  shape  [None, None, None, 32]
Final S

FileNotFoundError: ignored